# Indicizzatore di Ricette con Elasticsearch

Questo script Python indicizza i file `.txt` delle ricette in **Elasticsearch** utilizzando l'**analyzer italiano**,  
consentendo la ricerca full-text sui titoli e contenuti delle ricette.

Automaticamente:
- Crea o resetta l'indice (`index_recipes`)
- Indicizza in bulk tutte le ricette dalla cartella `files/`
- Supporta query `match` e `match_phrase`


In [60]:
from elasticsearch import Elasticsearch, helpers
import time
import os

# Connessione a Elasticsearch
ES_HOST = "http://localhost:9200"
es = Elasticsearch([ES_HOST])

# Elimina l'indice se esiste già
if es.indices.exists(index='index_recipes'):
    es.indices.delete(index='index_recipes')

# Definizione del mapping per l'indice
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text",
                      "analyzer": "italian",
                      "search_analyzer": "italian"},
            "content": {"type": "text", 
                        "analyzer": "italian",
                        "search_analyzer": "italian"}
        }
    }
}

def bulk_index(directory, index_name):
    actions = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            # Rimuove l'estensione .txt per ottenere il titolo
            title = os.path.splitext(filename)[0]
            
            action = {
                "_index": index_name,
                "_source": {
                    "title": title,
                    "content": text
                }
            }
            actions.append(action)

    # Indicizza i documenti in bulk con una sola chiamata a Elasticsearch
    if actions:
        helpers.bulk(es, actions)
        print(f"{len(actions)} documenti indicizzati nell'indice '{index_name}'.")
    else:
        print("Nessun file .txt trovato nella directory.")

def search(query, query_text=""):
   # Aggiunge highlight alla query con tag personalizzati
   query['highlight'] = {
       "fields": {
           "title": {},
           "content": {"fragment_size": 200, "number_of_fragments": 1}
       },
       "pre_tags": ["<<<HIGHLIGHT>>>"],
       "post_tags": ["<<<ENDHIGHLIGHT>>>"]
   }
   
   res = es.search(index='index_recipes', body=query)
   total = res['hits']['total']['value']
   print(f"\n{'='*80}")
   if query_text:
       print(f"Risultati della ricerca per la query \'{query_text}\': {total} ricette trovate")
   else:
       print(f"Risultati della ricerca: {total} ricette trovate")
   print(f"{'='*80}\n")
   
   # Codici colore ANSI
   YELLOW = '\033[93m'
   RESET = '\033[0m'
   
   for i, hit in enumerate(res['hits']['hits'], 1):  # Mostra tutti i risultati
    res_doc = hit['_source']
    
    # Usa highlight se disponibile, altrimenti usa il testo normale
    if 'highlight' in hit:
        title = hit['highlight'].get('title', [res_doc['title']])[0]
        if 'content' in hit['highlight']:
            content_preview = hit['highlight']['content'][0] + "..."
        else:
            content_preview = res_doc['content'][:200] + "..." if len(res_doc['content']) > 200 else res_doc['content']
    else:
        title = res_doc['title']
        content_preview = res_doc['content'][:200] + "..." if len(res_doc['content']) > 200 else res_doc['content']
    
    # Sostituisce i tag di highlight con colori
    title = title.replace("<<<HIGHLIGHT>>>", YELLOW).replace("<<<ENDHIGHLIGHT>>>", RESET)
    content_preview = content_preview.replace("<<<HIGHLIGHT>>>", YELLOW).replace("<<<ENDHIGHLIGHT>>>", RESET)
    
    print(f"{i}. Titolo: {title}")
    print(f"   Contenuto: {content_preview}\n")

def parse_and_search(query):
    """
    Analizza la query utente ed esegue la ricerca Elasticsearch appropriata.
    
    Args:
        query (str): Input utente nel formato 'campo:termine' o 'campo:"frase esatta"'
                     oppure solo 'termine' per multi_match su entrambi i campi
                     Campi supportati: 'title', 'content'
    
    Returns:
        None: Stampa i risultati direttamente o messaggi di errore
    """
    # Controlla se è specificato un campo
    if ":" not in query:
        # Nessun campo specificato, usa multi_match su title e content
        content_query = query.strip()
        if content_query.startswith('"') and content_query.endswith('"'):
            phrase = content_query.strip('"')
            body = {
                "query": {
                    "multi_match": {
                        "query": phrase,
                        "fields": ["title", "content"],
                        "type": "phrase"
                    }
                },
                "size": 10000
            }
        else:
            body = {
                "query": {
                    "multi_match": {
                        "query": content_query,
                        "fields": ["title", "content"]
                    }
                },
                "size": 10000
            }
        search(body, query)
        return
    
    type_query, content_query = query.split(":", 1)
    type_query = type_query.strip().lower()
    content_query = content_query.strip()

    # Costruisce il body della query per il campo specifico
    if type_query in ["title", "content"]:
        if content_query.startswith('"') and content_query.endswith('"'):
            phrase = content_query.strip('"')
            body = {"query": {"match_phrase": {type_query: phrase}}, "size": 10000}
        else:
            body = {"query": {"match": {type_query: content_query}}, "size": 10000}
        search(body, query)
    else:
        print("Campo sconosciuto. Usa 'title:' o 'content:'.")


## Misurazione del Tempo di Indicizzazione

Questo snippet misura quanto tempo impiega il processo di indicizzazione.


In [61]:
# Indicizza i documenti dalla directory 'files'
before = time.time()
es.indices.create(index='index_recipes', body=mapping)
bulk_index("files", "index_recipes")
after = time.time()
print(f"L'indicizzazione ha richiesto {after - before} secondi")


5939 documenti indicizzati nell'indice 'index_recipes'.
L'indicizzazione ha richiesto 1.044405221939087 secondi


## Query di Ricerca Interattiva (con risultati illimitati)

Questo snippet permette all'utente di inserire una query di ricerca in formato naturale e costruisce  
dinamicamente una query Elasticsearch, supportando:
- `multi_match` (cerca in titolo e contenuto)
- `match` (ricerca su campo singolo)
- `match_phrase` (ricerca frase esatta)


In [65]:
# Chiede all'utente di inserire una query
print("\nInserisci la tua query di ricerca usando uno dei seguenti formati:")
print('  • tiramisu (cerca in titolo e contenuto)')
print('  • title: tiramisu (cerca solo nel titolo)')
print('  • content: "burro e salvia" (frase esatta nel contenuto)')
print('  • content: banane (cerca solo nel contenuto)')
print("Usa le virgolette per cercare una frase esatta.\n")

query = input("Ricerca → ").strip()

# Usa la funzione refactorizzata
parse_and_search(query)



Inserisci la tua query di ricerca usando uno dei seguenti formati:
  • tiramisu (cerca in titolo e contenuto)
  • title: tiramisu (cerca solo nel titolo)
  • content: "burro e salvia" (frase esatta nel contenuto)
  • content: banane (cerca solo nel contenuto)
Usa le virgolette per cercare una frase esatta.


Risultati della ricerca per la query '"spaghetti alle vongole"': 4 ricette trovate

1. Titolo: Spaghetti vongole e nduja
   Contenuto: Ora gli spaghetti alle vongole e nduia sono pronti da servire...

2. Titolo: Spaghetti alle vongole
   Contenuto: Per preparare gli spaghetti alle vongole, cominciate dalla Assicuratevi che non ci siano gusci rotti o vuoti, andranno scartati....

3. Titolo: Spaghetti alle vongole e pomodorini
   Contenuto: Per preparare gli spaghetti alle vongole e pomodorini prima di tutto assicuratevi che non ci siano gusci rotti o aperti, andranno scartati....

4. Titolo: Spaghetti alle vongole con fiori di zucca e zafferano
   Contenuto: Per preparare gli spagh

## 10 query di test per l'analyzer italiano

Queste query testano **specificamente** i componenti della pipeline dell'analyzer italiano:

1. **italian_elision** → rimozione elisioni (l'origano → origano)
2. **light_italian stemmer** → plurali (mirtillo/mirtilli)
3. **lowercase** → non sensibile a maiuscole/minuscole (MOZZARELLA)
4. **italian_stop** → rimozione stopwords (della, di, il)
5. **light stemming** → preserva distinzioni semantiche (pomodoro ≠ pomodorino)
6. **genere** → variazioni maschile/femminile (fritto/fritta)
7. **elision + stop** → combinazione filtri (dell'uva → uva)
8. **stemming verbi** → participi passati (gratinato/gratinata)
9. **phrase query** → stopwords preservate in frasi esatte
10. **distinzioni semantiche** → ciliegia ≠ ciliegina (no over-stemming)


In [47]:
# Test 1: Elision filter - "l'origano" diventa "origano"
query = "l'origano"
parse_and_search(query)



Risultati della ricerca per la query 'l'origano': 163 ricette trovate

1. Titolo: Focaccia con pomodorini e origano
   Contenuto: La vostra focaccia con pomodorini e origano è pronta per essere gustata...

2. Titolo: Caprese sfiziosa con salsa all'origano
   Contenuto: Per preparare la caprese sfiziosa con salsa all’origano cominciate preparando la salsa all’origano....

3. Titolo: Piadina con pomodori secchi e mozzarella di bufala
   Contenuto: Procedete allo stesso modo per preparare l'altra e gustate la vostra piadina con pomodori secchi, bufala e origano...

4. Titolo: Roselline di melanzane
   Contenuto: melanzane, poi eliminate le due estremità e tagliatele per il lato corto a fettine sottili di circa 5 millimetri di spessore il sale Trascorso questo tempo, sfornate le melanzane Aromatizzate con l’origano...

5. Titolo: Filetto di platessa alla sorrentina
   Contenuto: Tritate l’origano versate la farina distribuendola su un vassoio aggiungete l’origano tritato il sale Passate i

In [49]:
# Test 2: Light stemming - "mirtillo" vs "mirtilli" (devono matchare)
query = 'mirtillo'
parse_and_search(query)


Risultati della ricerca per la query 'mirtillo': 60 ricette trovate

1. Titolo: Torta ai mirtilli
   Contenuto: e servire la vostra torta ai mirtilli....

2. Titolo: Confettura di ribes rosso e mirtilli
   Contenuto: La vostra confettura di ribes rosso e mirtilli è pronta...

3. Titolo: Muffin carote e mirtilli
   Contenuto: su ogni muffin (in totale vi serviranno circa 35 g di mirtilli) lasciate intiepidire i vostri muffin di carote e mirtilli prima di servirli!...

4. Titolo: Biscotti ai mirtilli
   Contenuto: Sfornate i biscotti ai mirtilli, fateli intiepidire su una gratella e poi serviteli...

5. Titolo: Flan di Bettelmatt
   Contenuto: qualche mirtillo fresco...

6. Titolo: Aspic ai frutti di bosco
   Contenuto: Per realizzare l’aspic ai frutti di bosco iniziate lavando e asciugando i mirtilli A parte in una ciotola e ponete i fogli di gelatina in acqua fredda per 10 minuti Scolate bene con un colino i fogli di...

7. Titolo: Gingerbread bundt cake
   Contenuto: In una ciotola c

In [55]:
# Test 3: Lowercase filter - test case-insensitive
query = 'MOZZARELLA'
parse_and_search(query)



Risultati della ricerca per la query 'MOZZARELLA': 150 ricette trovate

1. Titolo: Mozzarella ripiena
   Contenuto: Per realizzare le mozzarelle ripiene per prima cosa ritagliate la calotta di ciascuna mozzarella così da ottenere un incavo riducete a cubetti le calotte e teneteli da parte Ponete le mozzarelle scavate...

2. Titolo: Mozzarella fritta
   Contenuto: Ora potete servire i vostri bocconcini di mozzarella fritta ben caldi...

3. Titolo: Rotolo di mozzarella farcito
   Contenuto: Per realizzare il rotolo di mozzarella farcito, iniziate ponendo la mozzarella in una ciotola (scieglietene una poco più grande della mozzarella) con la propria acqua di conservazione e aggiungete altra...

4. Titolo: Spaghetti quadrati con sugo fresco e mozzarella
   Contenuto: la mozzarella...

5. Titolo: Mozzarella ripiena con cime di rapa
   Contenuto: DOP creando un buco a cono Estraete il pezzo di mozzarella Riempite la mozzarella DOP con le cime di rapa Trasferite in un piatto da portata Aggiu

In [ ]:
# Test 4: Light stemming 1 - "pomodoro" vs "pomodorino" (NON devono matchare)
query = 'pomodorino'
parse_and_search(query)



Risultati della ricerca per la query 'pomodorino': 402 ricette trovate

1. Titolo: Linguine con erbe miste, pomodorini e olio piccante
   Contenuto: Per preparare le linguine con erbe miste, pomodorini e olio piccante iniziate lavando con cura pomodorini e tagliando ogni pomodorino in quattro spicchi....

2. Titolo: Pomodorini confit
   Contenuto: Per preparare i pomodorini confit, iniziate lavando i pomodorini sotto acqua corrente Asciugateli con un canovaccio o carta da cucina Ora disponete i pomodorini tagliati su una leccarda ricoperta di carta...

3. Titolo: Schiacciate con crema al basilico
   Contenuto: amalgamato decorate per ultimo con i pomodorini confit e servite....

4. Titolo: Spaghetti integrali con pomodorini confit, feta e pesto leggero di rucola
   Contenuto: Per preparare gli spaghetti integrali con pomodorini confit, feta e pesto di rucola cominciate occupandovi dei pomodorini....

5. Titolo: Bucatini con pomodorini confit e cubetti di pancetta
   Contenuto: Per pre

In [ ]:
# Test 5: Light stemming 2
# NOTA: Questo può generare false positive, ad esempio cercando "ciliegina" (del cocktail)
# vengono trovati anche i "pomodorini ciliegini" perché condividono lo stesso stem
query = 'ciliegina'
parse_and_search(query)



Risultati della ricerca per la query 'ciliegina': 25 ricette trovate

1. Titolo: Pasta con stracchino, bresaola e ciliegini
   Contenuto: Per realizzare la pasta con stracchino, bresaola e ciliegini per prima cosa ponete sul fuoco una pentola colma di acqua, portatela al bollore e a quel punto salate....

2. Titolo: Penne alla crudaiola
   Contenuto: Per preparare le penne alla crudaiola lavate e tagliate a quarti i pomodorini ciliegino Spuntate, lavate e tagliate a rondelline sottili le zucchine novelle In una ciotola mettete i pomodorini, i funghi...

3. Titolo: Noodles di zucchine e carote con tonno e pomodoro
   Contenuto: striscioline Sovrapponete due strisce per volta, ripiegatele su stesse e poi ritagliate delle listarelle sottili Trasferite le carote e le zucchine all’interno di un ampia ciotola Lavate i pomodorini ciliegino...

4. Titolo: Verza alla salentina
   Contenuto: UniTe i pomodorini ciliegini tagliati a metà la verza scolata salate e coprite con acqua Cuocete per 30 

In [ ]:
# Test 6: Genere - "fritto/fritta/fritti/fritte" (devono matchare)
query = 'fritto'
parse_and_search(query)


In [ ]:
# Test 7: Elision + stopwords - "dell'uva" → "uva"
query = 'uva'
parse_and_search(query)


In [ ]:
# Test 8: Stemming su verbi - "gratinato/gratinata/gratinati" (root: gratin)
query = 'gratinato'
parse_and_search(query)


In [ ]:
# Test 9: Phrase query con stopwords - "al forno" (stopwords preservate in phrase)
query = '"al forno"'
parse_and_search(query)


In [ ]:
# Test 10: Distinzione semantica - "ciliegia" NON deve matchare "ciliegina"
query = 'ciliegia'
parse_and_search(query)
